In [2]:
TRAINING_DATASET_SOURCE = '../datasets/training_data.csv'  # Since we are one directory up, we should go down one directory to import the datasets
TEST_DATASET_SOURCE = '../datasets/test_data.csv'

In [3]:
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from pandas import DataFrame
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.preprocessing import OrdinalEncoder

train_df = pd.read_csv(TRAINING_DATASET_SOURCE)
test_df = pd.read_csv(TEST_DATASET_SOURCE)

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

features = train_df.drop(['incidents'], axis=1)
target = train_df['incidents'].map({'None': 0, 'Low': 1, 'Medium': 2, 'High': 3, 'Very_High': 4})


X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=1234)

In [5]:
X_train

,city_name,magnitude_of_delay,delay_in_seconds,affected_roads,record_date,luminosity,avg_temperature,avg_atm_pressure,avg_humidity,avg_wind_speed,avg_precipitation,avg_rain
3700,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101",2021-11-18 22:00,DARK,11.0,1025.0,67.0,2.0,0.0,Sem Chuva
894,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101",2021-09-27 20:00,DARK,18.0,1023.0,93.0,0.0,0.0,Sem Chuva
2322,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101",2021-11-17 23:00,DARK,11.0,1024.0,63.0,2.0,0.0,Sem Chuva
4592,Guimaraes,UNDEFINED,0,"N101,N101",2021-12-31 02:00,DARK,9.0,1023.0,74.0,2.0,0.0,Sem Chuva
3711,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101",2021-11-13 02:00,DARK,9.0,1019.0,74.0,1.0,0.0,Sem Chuva
...,...,...,...,...,...,...,...,...,...,...,...,...
664,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101,",2021-06-25 22:00,DARK,19.0,1016.0,58.0,0.0,0.0,Sem Chuva
3276,Guimaraes,UNDEFINED,0,",",2021-12-26 02:00,DARK,12.0,1009.0,93.0,0.0,0.0,Sem Chuva
1318,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101",2021-12-04 03:00,DARK,11.0,1024.0,94.0,1.0,0.0,Sem Chuva
723,Guimaraes,UNDEFINED,0,"N101,N101,N101,N101,N101",2021-10-14 00:00,DARK,17.0,1019.0,32.0,2.0,0.0,Sem Chuva


In [6]:
y_train

3700    0
894     0
2322    0
4592    0
3711    0
       ..
664     1
3276    0
1318    1
723     0
2863    4
Name: incidents, Length: 4000, dtype: int64

In [7]:
categorical_mask = (train_df.dtypes=='object')
categorical_columns = train_df.columns[categorical_mask].tolist()
num_cols = train_df.select_dtypes(include=['int64','float64']).columns.tolist()
oe_cols = [c for c in categorical_columns if train_df[c].nunique()>5]
ohe_cols = [c for c in categorical_columns if train_df[c].nunique()<=5]
len(oe_cols), len(ohe_cols), len(num_cols)

(2, 5, 6)

In [8]:
ohe_unique_list = [train_df[c].unique().tolist() for c in ohe_cols]
oe_unique_list = [train_df[c].unique().tolist() for c in oe_cols]
ohe = OneHotEncoder(categories=ohe_unique_list)
oe = OrdinalEncoder(categories=oe_unique_list)
imp = SimpleImputer(strategy='constant', fill_value=0)

In [9]:
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectKBest, RFE, f_classif

preprocess = make_column_transformer(
    (oe, oe_cols),
    (ohe, ohe_cols),
    (imp, num_cols),
    remainder='passthrough'
)
estimator = XGBClassifier(learning_rate=0.05, max_depth=3, n_estimators=2500, random_state=1234)
fs = SelectKBest(score_func=f_classif, k=5)
selector = RFE(estimator, n_features_to_select=5, step=1)
steps = [
    ('preprocess', preprocess),
    ('select', fs),
    ('clf', estimator)
]
pipeline = Pipeline(steps)

In [10]:
from sklearn.metrics import accuracy_score

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
pred_df = pd.DataFrame({'y': y_test,'y_pred': y_pred})
gini = 2*accuracy_score(y_test, y_pred)-1

ValueError: A given column is not a column of the dataframe